In [ ]:
import math
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pysrt
from geopy import distance

# Prüfung der Sektorgrenzen von Sektorenfeuern mittels Drohnenaufnahmen

## Einleitung

Das folgende Jupyter Notebook beschäftigt sich mit der Überprüfung der Sektorgrenzen von Sektorenfeuern anhand von Drohnenaufnahmen. Bei Sektorenfeuern handelt es sich um Seeschifffahrtszeichen, mit unterschiedlichen Kennungen oder Farben. So markiert ein Leitfeuer wie aus Abbildung 1 mittels des weißen Sektors die optimale Fahrlinie im Fahrwasser, während der Rote und Grüne Sektor die Notwendigkeit zum Korrigieren des Kurses aufzeigen.[[1]](https://doi.org/10.24053/9783739882161)

<p align="center">
<img src="./abbildungen/leitfeuer.png" width="700">
<br>
Abbildung 1: Leitfeuer 
</p>

Diese Sektorenfeuer sind in einigen Fällen auf eine hohe Präzision angewiesen, da diese über eine hohe Tragweite verfügen und bereits geringe Abweichungen in einer entsprechend großen Entfernung, wie beispielsweise 30 Seemeilen im Falle des Leuchtturms Campen, große Auswirkungen haben.[[2]](https://www.deutsche-leuchtfeuer.de/nordsee/campen.html)

Für die Erarbeitung von Lösungswegen für eine solche Überprüfung werden in diesem Notebook Drohnenaufnahmen des Leuchtfeuers Wybelsum verwendet. Bei diesem Handelt es sich um ein Quermarkenfeuer mit den Sektoren W 295°-320°, R -024°, W -049° an der Position N53°20'10" E07°06'20" mit einer Feuerhöhe von 16 m.[[3]](https://www.deutsche-leuchtfeuer.de/binnen/ems/wybelsum.html) In Abbildung 2 ist ein Kartenausschnitt des Leuchtfeuers und dessen Sektorgrenzen zu sehen.[[4]](https://map.openseamap.org/)

<p align="center">
<img src="./abbildungen/wybelsum.png" width="700">
<br>
Abbildung 2: Quermarkenfeuer Wybelsum
</p>

Die zur Verfügung gestellten Aufnahmen bestehen aus mehreren Abflügen des Sektorenfeuers bei Nacht, welche sich in Höhe und Entfernung zum Feuer unterscheiden. Die zu den Aufnahmen gehörenden Metadaten liegen als Untertitel in den `.SRT` Dateien bei. 

In [ ]:
!wget -O data/Flug_1.MP4 https://mux.hs-emden-leer.de/stdaw/20240613_nachtaufnahmen_sektorenfeuer/DJI_20240613232223_0002_V.MP4
!wget -O data/Flug_1.SRT https://mux.hs-emden-leer.de/stdaw/20240613_nachtaufnahmen_sektorenfeuer/DJI_20240613232223_0002_V.SRT

!wget -O data/Flug_2.MP4 https://mux.hs-emden-leer.de/stdaw/20240613_nachtaufnahmen_sektorenfeuer/DJI_20240613232811_0003_V.MP4
!wget -O data/Flug_2.SRT https://mux.hs-emden-leer.de/stdaw/20240613_nachtaufnahmen_sektorenfeuer/DJI_20240613232811_0003_V.SRT


### Projektrahmen

Das Projekt wird im Rahmen des Moduls "Spezielle Themen der Datenwissenschaft" bei Prof. Carsten Koch an der Hochschule Emden-Leer durchgeführt. Die Idee des Projektes kam dabei von dem Wasserstraßen- und Schifffahrtsamt in Emden. Die Umsetzung erfolgt durch die Studenten Adrian Schiel(7022935), Mirko Labitzke(7021691) und Tarek Harms(7022221) im Sommersemester 2024. Das Projekt wird neben Prof. Carsten Koch noch von den studentischen Mitarbeitern Tilman Leune und Malte Czesnik betreut. Die Drohnenaufnahmen wurden ebenfalls von diesen aufgenommen und bereitgestellt.

## Analyse

Die Analyse zu diesem Projekt besteht aus der Sichtung der Drohnenaufnahmen sowie der Recherche nach weiterer Literatur, in denen eine ähnliche Problematik behandelt wird. Zu den für die Recherche verwendeten Schlagworten gehörten "visual", "UAV" (unmanned aerial vehicle) und "ATON" (aids to navigation). Unter diesen Schlagworten lassen sich allerdings keine Ergebnisse finden, welche für dieses Projekt gewinnbringend sind. Aus dem Bereich der Ampelerkennung lässt sich das Paper "Traffic light detection with color and edge information"[[5]](https://ieeexplore.ieee.org/document/5234518) finden, in dem die Erkennung von Ampellichtern anhand der Farbe und Form realisiert wird.  
Bei der Sichtung der Videoaufnahmen ist zu erkennen, dass der sich im Hintergrund befindliche Emder Hafen viele weitere Lichtquellen ins Bild bringt. Für eine bessere Handhabe der Aufnahmen wird diese zunächst in kleinere Ausschnitte aufgeteilt. Hierbei werden 5 Sekunden lange Ausschnitte des Videos und Untertitels in denen ein Wechsel der Sektoren auftritt manuell mithilfe des Kommandozeilenwerkzeugs `ffmpeg` ausgeschnitten.


In [ ]:
!ffmpeg -i data/Flug_1.MP4 -ss 00:00:15 -to 00:00:20 -c copy data/Flug_1_part_1.MP4
!ffmpeg -i data/Flug_1.SRT -ss 00:00:15 -to 00:00:20 -c copy data/Flug_1_part_1.SRT

!ffmpeg -i data/Flug_1.MP4 -ss 00:00:43 -to 00:00:48 -c copy data/Flug_1_part_2.MP4
!ffmpeg -i data/Flug_1.SRT -ss 00:00:43 -to 00:00:48 -c copy data/Flug_1_part_2.SRT

!ffmpeg -i data/Flug_1.MP4 -ss 00:01:38 -to 00:01:43 -c copy data/Flug_1_part_3.MP4
!ffmpeg -i data/Flug_1.SRT -ss 00:01:38 -to 00:01:43 -c copy data/Flug_1_part_3.SRT

!ffmpeg -i data/Flug_1.MP4 -ss 00:02:57 -to 00:03:02 -c copy data/Flug_1_part_4.MP4
!ffmpeg -i data/Flug_1.SRT -ss 00:02:57 -to 00:03:02 -c copy data/Flug_1_part_4.SRT

!ffmpeg -i data/Flug_1.MP4 -ss 00:03:43 -to 00:03:48 -c copy data/Flug_1_part_5.MP4
!ffmpeg -i data/Flug_1.SRT -ss 00:03:43 -to 00:03:48 -c copy data/Flug_1_part_5.SRT

In [ ]:
!ffmpeg -i data/Flug_2.MP4 -ss 00:00:01 -to 00:00:06 -c copy data/Flug_2_part_1.MP4
!ffmpeg -i data/Flug_2.SRT -ss 00:00:01 -to 00:00:06 -c copy data/Flug_2_part_1.SRT

### Verwendete Bibliotheken

- `cv2`: OpenCV, eine Bibliothek zur Echtzeit-Computer-Vision. (`4.10.0.84`) 
- `matplotlib.pyplot`: Eine Bibliothek für die Erstellung von Diagrammen und Visualisierungen.
- `numpy`: Eine Bibliothek zur numerischen Berechnung in Python. (`1.26.4`)
- `pysrt`: Eine Bibliothek zum Lesen, Schreiben und Bearbeiten von `.srt`-Untertiteldateien. (`1.1.2`)
- `geopy`: Eine Bibliothek zur Berechnung von Entfernungen zwischen zwei Punkten angegeben in Längen- und Breitengraden. (`2.4.1`)


### Verwendete Pfade

- `video_path`: Pfad zum Eingabevideo, welches verarbeitet bzw. analysiert werden soll
- `srt_path`: Pfad zur passenden Untertiteldatei zu7m Eingabevideo im .srt-Format
- `output_video`:   Pfad zum Ausgabevideo des Jupiter-Notebooks

In [ ]:
video_path = 'data/Flug_1_part_1.MP4'
srt_path = 'data/Flug_1_part_1.SRT'
output_video = 'output_video_with_subtitles_part_1_hsv.mp4'

## Funktion: `get_all_frames`

Diese Funktion lädt alle Frames des Eingabevideos und speichert sie als RGB-Bilder in einem Array.

### Ablauf:
1. **Video einlesen**: Öffnet das Video und ermittelt die Eigenschaften wie Breite, Höhe und Anzahl der Frames.
2. **Frames speichern**: Liest jedes Frame ein, konvertiert es in den RGB-Farbraum und speichert es in einem Array (`buf`).
3. **Ressourcen freigeben**: Schließt das Video nach dem Einlesen aller Frames.
4. **Ergebnis**: Gibt das Array mit den konvertierten Frames zurück.

In [ ]:
def get_all_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret, frame = cap.read()

        buf[fc] = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        fc += 1


    cap.release()

    return buf

Das Eingabevideo wird eingelesen, danach wird das erste und das letzte Bild des Arrays angezeigt.

In [ ]:
frames = get_all_frames(video_path)
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
axs[0].imshow(frames[0])
axs[1].imshow(frames[-1])
plt.show()

Die Bilder veranschaulichen die Änderung während der Aufnahme, dabei befindet sich das Sektorenfeuer im gesamten Ausschnitt ungefähr mittig, weiter links und rechts im Bild auf etwa gleicher Höhe befinden sich weitere Lichtquellen, die auf den ersten Blick eine ähnliche Farbe wir das weiße Sektorenfeuer aufweisen. 

In [ ]:
def plot_hsv(frame_hsv, fig, pos, channel, label, cmap='hsv'):
    ax = fig.add_subplot(pos, projection='3d')
    H = frame_hsv[:, :, channel]    
    X, Y = np.meshgrid(range(frame_hsv.shape[1]), range(frame_hsv.shape[0]))
    ax.plot_surface(X, Y, H, cmap=cmap)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_box_aspect(aspect=None, zoom=0.8)
    ax.set_zlabel(label)

In [ ]:
frames_cropped = frames[:, 1175:1200, 1650:1900, :]

fig = plt.figure(figsize = (20, 10))

axs1 = fig.add_subplot(241)
axs2 = fig.add_subplot(245)
axs1.imshow(frames_cropped[82])
axs2.imshow(frames_cropped[85])

frame_hsv = cv2.cvtColor(frames_cropped[82], cv2.COLOR_RGB2HSV)
plot_hsv(frame_hsv, fig, 242, 0, "Hue")
frame_hsv = cv2.cvtColor(frames_cropped[85], cv2.COLOR_RGB2HSV)
plot_hsv(frame_hsv, fig, 246, 0, "Hue")

frame_hsv = cv2.cvtColor(frames_cropped[82], cv2.COLOR_RGB2HSV)
plot_hsv(frame_hsv, fig, 243, 1, "Saturation", cmap='hot')
frame_hsv = cv2.cvtColor(frames_cropped[85], cv2.COLOR_RGB2HSV)
plot_hsv(frame_hsv, fig, 247, 1, "Saturation", cmap='hot')

frame_hsv = cv2.cvtColor(frames_cropped[82], cv2.COLOR_RGB2HSV)
plot_hsv(frame_hsv, fig, 244, 2, "Value", cmap='hot')
frame_hsv = cv2.cvtColor(frames_cropped[85], cv2.COLOR_RGB2HSV)
plot_hsv(frame_hsv, fig, 248, 2, "Value", cmap='hot')
plt.show()

Die hier gezeigten Abbildungen zeigen 3D Plots der einzelnen HSV-Kanäle. Hierbei wird bewusst ein schmaler Streifen aus dem Bild verwendet, um somit den Rand des Feuers besser zu erkennen. Während das weiße Feuer einen klar zu unterscheidenden Hue wert gegenüber dem Hintergrund aufweist, geht dieser beim roten Feuer im Rauschen etwas unter. Ebenso nimmt das Zentrum des roten Feuers ähnliche H Werte wie das weiße Feuer an. In der Helligkeit und Sättigung hebt sich das Feuer gegenüber dem benachbarten Hintergrund ab. 


## Konzept

Die Erkennung und Prüfung des Sektorenfeuers wird in drei Schritte unterteilt. Zunächst wird die Position des Feuers auf dem Video ausgemacht, um den weiterzuverarbeitenden Ausschnitt des Videos einzugrenzen. Es folgt eine Erkennung der Sektorgrenze anhand der aufeinanderfolgenden Bildausschnitte. Zuletzt wird anhand der Position des Leuchtturms und der Position der Drone berechnet, wie weit die Sektorgrenze vom vorgegeben Kurs abweicht. 


### Position des Leuchtfeuers

Um die genaue Position des Leuchtfeuers innerhalb eines Videos zu ermitteln, ist es entscheidend, die relevanten Bildbereiche präzise zu identifizieren und zu analysieren. Für diese Identifizierung der Bildbereiche sind zwei Vorgehensweisen vorgesehen. Die erste orientiert sich an dem Vorgehen aus dem Paper Traffic light detection with color and edge information[[5]](https://ieeexplore.ieee.org/document/5234518). In diesem werden die RGB-Werte eines Frames normalisiert und anhand von Filtern das Leuchtsignal hervorgehoben. 
Das zweite Vorgehen ähnelt dem aus dem Paper, allerdings werden hier im HSV-Farbraum gefiltert. Dazu werden die einzelnen Frames des Videos untersucht, um potenzielle Lichtquellen anhand ihrer charakteristischen Farben zu erkennen. Diese Farben, in diesem Fall im roten und weißen Spektrum, werden durch entsprechende Farbmasken im HSV-Farbraum hervorgehoben. Durch die Ermittlung der Konturen der identifizierten Farbbereiche und die Bestimmung der größten zusammenhängenden Objekte lassen sich die Position und Ausdehnung des Feuers in jedem Frame des Videos in Form von Bounding Boxes präzise festhalten.

### Bestimmung der Sektorgrenze

Die Bestimmung der Sektorgrenze erfolgt anhand der für den Bildausschnitt berechneten Durchschnittswerte der Farbkanäle der einzelnen Frames. Für die einzelnen Kanäle wird geprüft, ob und wie deutlich eine Sektorgrenze zu erkennen ist. Hierzu werden die einzelnen Kanäle der Farbräume RGB und HSV verwendet. 


### Prüfen des Kurses

Die Prüfung des Kurses der Sektorgrenze erfolgt mittels der Position der Drohne an der ermittelten Sektorgrenze und der Position des Feuers. Anhand dieser Parameter wird der Kurs von der Drohne zum Feuer berechnet. Dieser berechnete Kurs wird daraufhin mit dem Soll-Wert verglichen. 

## Umsetzung

### Normalisieren und filtern im RGB-Farbraum

Das Verfahren aus dem erwähnten Paper sieht vor zunächst die Farbwerte des Frames zu normalisieren. Durch diese Normalisierung wird eine bessere Vergleichbarkeit der Farben untereinander erzielt[[7]](https://www.sciencedirect.com/science/article/pii/S0031320316301510). Diese Normalisierung wird in der Funktion `normalize_rgb_frame` implementiert. 

In [ ]:
def normalize_rgb_frame(frame_rgb):
    frame_rgb = np.array(frame_rgb)    
    frame_normalized_rgb = np.zeros_like(frame_rgb)
    sums = np.sum(frame_rgb[:, :], axis=2)

    frame_normalized_rgb[:, :, 0] = frame_rgb[:, :, 0] / sums * 255
    frame_normalized_rgb[:, :, 1] = frame_rgb[:, :, 1] / sums * 255
    frame_normalized_rgb[:, :, 2] = frame_rgb[:, :, 2] / sums * 255
    
    return frame_normalized_rgb


normalized_frame_first = normalize_rgb_frame(frames[0])
normalized_frame_last = normalize_rgb_frame(frames[-1])

fig, axs = plt.subplots(1, 2, figsize=(16, 8))
axs[0].imshow(normalized_frame_first)
axs[1].imshow(normalized_frame_last)
plt.show()

Die obere Abbildung zeigt den ersten und letzten Frame mit normalisierten RGB-Werten. Der im ersten Frame zu sehende rote Sektor hebt sich vom Hintergrund ab. Der im letzten Frame zu sehende weiße Sektor hingegen weniger deutlich. Nach der Normalisierung werden mittels der Funktion `get_filter_frame` Filter erstellt. Die Grenzwerte der Filter werden zunächst aus dem Paper übernommen.  

In [ ]:
def get_filter_frame(normalized_frame):    
    filter_R_1 = normalized_frame[:, :, 0] > 200
    filter_G_1 = normalized_frame[:, :, 1] < 150
    filter_B_1 = normalized_frame[:, :, 2] < 150
    
    filter_1 = filter_R_1 & filter_G_1 & filter_B_1

    filter_R_2 = normalized_frame[:, :, 0] > 200
    filter_G_2 = normalized_frame[:, :, 1] > 150
    filter_B_2 = normalized_frame[:, :, 2] < 150
    
    filter_2 = filter_R_2 & filter_G_2 & filter_B_2
    
    filter_R_3 = normalized_frame[:, :, 0] < 150
    filter_G_3 = normalized_frame[:, :, 1] > 240
    filter_B_3 = normalized_frame[:, :, 2] > 220
    
    filter_3 = filter_R_3 & filter_G_3 & filter_B_3
    
    return filter_1 | filter_2 | filter_3


filter_frame_first = get_filter_frame(normalized_frame_first)
filter_frame_last = get_filter_frame(normalized_frame_last)

fig, axs = plt.subplots(1, 2, figsize=(16, 8))
axs[0].imshow(filter_frame_first)
axs[1].imshow(filter_frame_last)
plt.show()

Die Abbildungen zeigen, dass der aus dem Paper übernommene Filter sich nicht nahtlos auf die Problemstellung dieser Arbeit anwenden lässt, weshalb die Filter im folgenden etwas angepasst werden.  

In [ ]:
def get_filter_frame(normalized_frame):
    filter_R_1 = normalized_frame[:, :, 0] > 125
    filter_G_1 = normalized_frame[:, :, 1] < 200
    filter_B_1 = normalized_frame[:, :, 2] < 200

    filter_1 = filter_R_1 & filter_G_1 & filter_B_1

    filter_R_2 = normalized_frame[:, :, 0] > 100
    filter_G_2 = normalized_frame[:, :, 1] > 75
    filter_B_2 = normalized_frame[:, :, 2] < 150

    filter_2 = filter_R_2 & filter_G_2 & filter_B_2

    filter_R_3 = normalized_frame[:, :, 0] < 150
    filter_G_3 = normalized_frame[:, :, 1] > 240
    filter_B_3 = normalized_frame[:, :, 2] > 220

    filter_3 = filter_R_3 & filter_G_3 & filter_B_3

    return filter_1 | filter_2 | filter_3


filter_frame_first = get_filter_frame(normalized_frame_first)
filter_frame_last = get_filter_frame(normalized_frame_last)

fig, axs = plt.subplots(1, 2, figsize=(16, 8))
axs[0].imshow(filter_frame_first)
axs[1].imshow(filter_frame_last)
plt.show()

Durch die Anpassung der filter hebt sich sowohl der rote als auch der weiße Sektor deutlich ab. Die Filter weisen nun allerdings eine solch hohe Empfindlichkeit auf, sodass auch das Hintergrundrauschen sichtbarer wird. Die Normalisierung und der Filter werden in der im folgenden erläuterten Funktion `get_Bounding_Boxes` angewendet.  


### Funktion: `get_Bounding_Boxes`

Diese Funktion versucht in dem Eingabevideo den Leuchtturm zu finden und passend um diesen für jeden Frame in dem Video ein Rechteck zu generieren. Das Array bounding_boxes wird zurückgeben und beinhaltet für jeden Frame im Video ein Koordinatenpaar für das jeweilige Rechteck, sowie die Höhe und Breite. Die Funktion sucht primär nach roten und weißen Flächen, dies müsste für Leuchttürme angepasst werden, bei denen andere Farben relevant sind.

#### Ablauf:
1. **Video laden**: Öffnet das Video und initialisiert eine Liste zur Speicherung der Bounding Boxes.
2. **Frames verarbeiten**: Jedes Frame wird auf den zentralen Bereich zugeschnitten und in den HSV-Farbraum konvertiert.
3. **Farberkennung**: Normalisierung des Frames und Erstellung der Maske
4. **Konturen finden**: Die größte Kontur in jedem Frame wird erkannt, und ihre Bounding Box wird gespeichert.
5. **Ergebnis**: Gibt eine Liste der Bounding Boxes für jedes Frame zurück.

In [ ]:
def get_Bounding_Boxes(video_path):
    # Video öffnen
    cap = cv2.VideoCapture(video_path)

    # Liste zur Speicherung der Bounding Box Positionen
    bounding_boxes = []

    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fc = 0
    ret = True
    while (fc < frameCount  and ret):
        ret, frame = cap.read()
        if not ret:
            break

        # Frame in den mittleren Bereich zuschneiden
        height, width, _ = frame.shape
        center_frame = frame[height//4: 3*height//4, width//4: 3*width//4]

        rgb_frame = cv2.cvtColor(center_frame, cv2.COLOR_BGR2RGB)
        normalized_rgb_frame = normalize_rgb_frame(rgb_frame)
        filtermaske = get_filter_frame(normalized_rgb_frame)
        filtermaske = (filtermaske.astype(np.uint8)) * 255

        # Konturen finden
        contours, _ = cv2.findContours(filtermaske, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Größtes Objekt finden und Position speichern
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest_contour)
            bounding_boxes.append([x + width//4, y + height//4, w, h])

        fc += 1
    return bounding_boxes

### Funktion: `get_subtitle_for_time`

Diese Funktion sucht die passenden Metadaten für einen bestimmten Zeitpunkt im Video.

### Ablauf:
1. **Zeiten berechnen**: Die Start- und Endzeit jedes Untertitels werden in Sekunden umgerechnet.
2. **Untertitel finden**: Es wird überprüft, ob der gegebene Zeitpunkt innerhalb der Zeitspanne eines Untertitels liegt.
3. **Ergebnis**: Gibt den Text die entsprechenden Metadaten zurück oder `None`, wenn kein Untertitel für diesen Zeitpunkt vorhanden ist.

In [ ]:
def get_subtitle_for_time(time_in_seconds, subs):
    for sub in subs:
        start_time = sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds + sub.start.milliseconds / 1000.0
        end_time = sub.end.hours * 3600 + sub.end.minutes * 60 + sub.end.seconds + sub.end.milliseconds / 1000.0
        if start_time <= time_in_seconds <= end_time:
            return sub.text
    return None

### Funktion: `get_all_frames_cropped`

Diese Funktion extrahiert und verarbeitet Frames aus einem Video, wobei sie auf einen zentralen Bereich um den Leuchtturm herum zugeschnitten werden. Optional kann Otsu's Methode zur Bildsegmentierung angewendet werden, wodurch Hintergrundrauschen keinen Einfluss mehr auf die Berechnung und Analyse hat. Dabei werden alle Hintergrundpixel auf 0 gesetzt, wodurch der Hintergrund schwarz wird.

### Ablauf:
1. **Bounding Boxes analysieren**: Berechnung des zentralen Bereichs für den Crop basierend auf den Bounding Boxes.
2. **Crop-Bereich festlegen**: Der Bereich wird auf eine Mindestgröße begrenzt und innerhalb der Bildgrenzen gehalten.
3. **Frames verarbeiten**: 
   - Jedes Frame wird auf den definierten Bereich zugeschnitten.
   - Optional: Otsu's Methode zur Trennung von Vorder- und Hintergrund.
   - Ermittlung des passenden Untertitels für das aktuelle Frame.
4. **Ergebnis**: Gibt die zugeschnittenen Frames und die dazugehörigen Untertitel als Arrays zurück.


In [ ]:
def get_all_frames_cropped(video_path: str, srt_path: str, bounding_boxes: np.ndarray, height: int, width: int, use_otsu: bool = False) -> np.ndarray:

    x_mean = int(np.mean(bounding_boxes[:, 0]) + np.mean(bounding_boxes[:, 2]) // 2)
    y_mean = int(np.mean(bounding_boxes[:, 1]) + np.mean(bounding_boxes[:, 3]) // 2)

    # Mindestgröße von 200x200px für den Crop-Bereich festlegen
    crop_width = max(width, int(np.mean(bounding_boxes[:, 2])))
    crop_height = max(height, int(np.mean(bounding_boxes[:, 3])))

    # Position so anpassen, dass der Punkt in der Mitte des Crop-Bereichs ist
    x_start = max(0, x_mean - crop_width // 2)
    y_start = max(0, y_mean - crop_height // 2)

    cap = cv2.VideoCapture(video_path)
    subs = pysrt.open(srt_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
      
    # Sicherstellen, dass der Crop-Bereich innerhalb des Bildes liegt
    if x_start + crop_width > frameWidth:
        x_start = frameWidth - crop_width
    if y_start + crop_height > frameHeight:
        y_start = frameHeight - crop_height

    subtitles = [None] * frameCount

    buf = np.empty((frameCount, height, width, 3), np.dtype('uint8'))
    fc = 0

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break
        # Zuschneiden des Frames auf den zentrierten Bereich
        cropped_frame = frame[y_start:y_start+crop_height, x_start:x_start+crop_width]

        frame_time = fc / fps

        subtitle = get_subtitle_for_time(frame_time, subs)
        subtitles[fc] = subtitle

        if use_otsu:
            # Otsu's Methode anwenden, um Vorder- und Hintergrund zu trennen
            gray = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)
            
            _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Erstellen des binären Bildes (Vordergrund = 1, Hintergrund = 0)
            binary_mask = mask // 255  # Umwandlung in 0 und 1
            # Anwenden der Maske auf das Originalbild, um den Hintergrund zu schwärzen
            filtered_frame = cropped_frame * np.expand_dims(binary_mask, axis=-1)

        else:
            filtered_frame = cropped_frame

        buf[fc] = cv2.cvtColor(filtered_frame, cv2.COLOR_BGR2RGB)
        fc += 1

    return buf, subtitles

Aufrufen der vorher definierten Methode und speichern in eine lokale Variabel.

In [ ]:
bounding_boxes = get_Bounding_Boxes(video_path)
frames_cropped, subtitles = get_all_frames_cropped(video_path, srt_path, np.array(bounding_boxes), 400, 400, False)


### Funktionen: `get_avg_hsv` und `get_avg_rgb`

Diese Funktionen berechnen den durchschnittlichen Farbwert eines Bildes im HSV- bzw. RGB-Farbraum.

#### `get_avg_hsv`:
1. **RGB zu HSV konvertieren**: Wandelt das Bild in den HSV-Farbraum um.
2. **Maske anwenden**: Verwendet eine Maske, um gültige Pixel auszuwählen.
3. **Ergebnis**: Gibt den durchschnittlichen HSV-Wert des Bildes zurück.

#### `get_avg_rgb`:
1. **Maske anwenden**: Erzeugt eine Maske, um schwarze Pixel auszuschließen.
2. **Ergebnis**: Gibt den durchschnittlichen RGB-Wert des Bildes zurück.

#### Iteration:
- **Averages berechnen**: Iteriert über alle Frames und berechnet die durchschnittlichen HSV- und RGB-Werte für jedes Frame, speichert diese in den Arrays `averages_hsv` und `averages_rgb`.


In [ ]:
def get_avg_hsv(rgb_image):
    hsv_image = rgb_image.astype('float32')/255
    hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_RGB2HSV)
    
    lower_limit = np.array([0, 0, 0])
    upper_limit = np.array([360, 255, 255])
    
    mask = cv2.inRange(hsv_image, lower_limit, upper_limit)
    
    return cv2.mean(hsv_image, mask)[:3]

def get_avg_rgb(rgb_image):
    mask = cv2.inRange(rgb_image, np.array([1, 1, 1]), np.array([255, 255, 255]))
    return cv2.mean(rgb_image, mask)[:3]


averages_hsv = np.zeros((frames_cropped.shape[0], frames_cropped.shape[3]), np.dtype('float32'))
averages_rgb = averages_hsv.copy()
for i in range(frames_cropped.shape[0]):
    averages_hsv[i] = get_avg_hsv(frames_cropped[i])
    averages_rgb[i] = get_avg_rgb(frames_cropped[i])


### Funktion: `getIndicies`

Diese Funktion ermittelt die Indizes der größten Gradienten also der größten absoluten Steigung in einem gegebenen Array. Dabei wird für jeden Kanal ein eigener Index berechnet und diese werden als Array zurückgegeben.

#### Ablauf:
1. **Gradienten berechnen**: Für jede der drei Spalten des `averages`-Arrays werden die Gradienten über eine festgelegte Anzahl von Schritten berechnet.
2. **Abfall identifizieren**: Der Index des größten Abfalls (bzw. des steilsten Anstiegs) in den Gradienten wird für jede Spalte bestimmt.
3. **Ergebnis**: Gibt eine Liste der Indizes mit den steilsten Gradienten für jede Spalte zurück.


In [ ]:
def getIndices(averages: np.ndarray) -> list:
    indices = []
    for i in range(3):
        n = 2 
        gradients = averages[n:, i] - averages[:-n, i]
        x = np.min(gradients)

        # Index des steilsten Abfalls finden
        x_index = np.argmax( np.abs(gradients))
        indices.append(x_index)
    return indices

### Funktion: `plot`

Diese Funktion visualisiert die Durchschnittswerte eines Arrays und markiert die bestimmten Indizes in den Diagrammen.

#### Ablauf:
1. **Subplots erstellen**: Erstellt drei separate Diagramme (eines für jede Spalte des `averages`-Arrays).
2. **Indizes markieren**: Zeichnet eine vertikale Linie an den angegebenen Indizes.
3. **Daten plotten**: Plottet die Durchschnittswerte der drei Spalten.
4. **Anzeigen**: Zeigt die Plots an.


In [ ]:
def plot(averages: np.ndarray, indices: list):

    fig, axs = plt.subplots(3, figsize=(8, 8))

    print(indices)

    axs[0].axvline(x = indices[0], color = 'b')
    axs[0].plot(averages[:, 0])
    axs[1].axvline(x = indices[1], color = 'b')
    axs[1].plot(averages[:, 1])
    axs[2].axvline(x = indices[2], color = 'b')
    axs[2].plot(averages[:, 2])
    plt.show()

Aufrufen der `plot` Methoden mit jeweils HSV-Werten und RGB-Werten. Dabei werden die jeweiligen Diagramme ausgegeben und die berechneten Indizes ausgegeben.

In [ ]:
plot(averages_hsv, getIndices(averages_hsv))
plot(averages_rgb, getIndices(averages_rgb))

### Code-Beschreibung

Dieser Code bestimmt den Wechselrahmen basierend auf durchschnittlichen Farbwerten und wählt das entsprechende Frame aus.

#### Ablauf:
1. **Indizes berechnen**: Die Funktion `getIndices` wird auf die durchschnittlichen RGB-Werte (`averages_rgb`) angewendet, um relevante Indizes zu ermitteln.
2. **Wechselrahmen bestimmen**: Berechnet den Mittelwert der Indizes und rundet ihn, um den Wechselrahmen (`switch_frame`) zu bestimmen.
3. **Frame auswählen**: Das Frame an der Position `switch_frame` wird aus `frames_cropped` extrahiert und als `frame` gespeichert.

In [ ]:
#switch_frame = int(np.round(np.mean(getIndices(averages_hsv))))
switch_frame = int(np.round(np.mean(getIndices(averages_rgb))))
frame = frames_cropped[switch_frame]

### Code-Beschreibung

Dieser Code zeigt das vorher bestimmte Bild an und fügt den Untertitel mit geografischen Koordinaten hinzu.

#### Ablauf:
1. **Bild anzeigen**: Erstellt eine Plotfigur und zeigt das ausgewählte Frame (`frame`) an.
2. **Untertitel extrahieren**: Extrahiert die geografischen Koordinaten (Breitengrad und Längengrad) aus dem Untertitel des `switch_frame`.
3. **Text hinzufügen**: Platziert den Untertitel mit den Koordinaten auf dem Bild, wobei der Text weiß ist und einen schwarzen Hintergrund hat.
4. **Anzeigen**: Zeigt das Bild mit dem überlagerten Text an.


In [ ]:
plt.figure(figsize=(16, 9))
plt.imshow(frame)
subtitle = str(subtitles[switch_frame]).split('[')
subtitle = [subtitle[9].split("latitude: ")[1][:~1], subtitle[10].split("longitude: ")[1][:~1]]
plt.text(5, 10, subtitle, color='white', fontsize=12, bbox=dict(facecolor='black', alpha=0.5))
plt.show()

### Funktion: `generate_video_with_subtitles`

Diese Funktion erstellt ein neues Video mit eingebetteten Metadaten und markierten Bounding Boxes. Dabei werden, sobald ein Wechsel der Farben bestimmt wurde, zusätzlich die Metadaten des bestimmten Wechselzeitpunkt-Bildes angezeigt. Die Bounding-Boxes ändern dabei ebenfalls die Farbe. 

#### Ablauf:
1. **Video einlesen**: Lädt das Eingabevideo und ermittelt die Eigenschaften (Breite, Höhe, FPS).
2. **VideoWriter initialisieren**: Bereitet den VideoWriter vor, um das bearbeitete Video zu speichern.
3. **Frames bearbeiten**:
   - Fügt Untertitel zu jedem Frame hinzu.
   - Markiert die Bounding Boxes..
   - Bei Erreichen des `switch_frame_number` wird der zusätzlicher Text eingeblendet.
4. **Video speichern**: Speichert die bearbeiteten Frames im Ausgabefile.
5. **Ressourcen freigeben**: Schließt die Video-Dateien.

In [ ]:
def generate_video_with_subtitles(input_video_path, output_video_path, switch_frame_number):
    # Video einlesen
    cap = cv2.VideoCapture(input_video_path)
    
    # Videoeigenschaften ermitteln
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # VideoWriter zum Speichern des neuen Videos initialisieren
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Video durchlaufen und bearbeiten
    frame_counter = 0
    switch_subtitle = str(subtitles[switch_frame_number+1]).split('[')
    switch_subtitle = [switch_subtitle[9].split("latitude: ")[1][:~1], switch_subtitle[10].split("longitude: ")[1][:~1]]

    while cap.isOpened():
        try:
            ret, frame = cap.read()
            if not ret:
                break

            subtitle = str(subtitles[frame_counter+1]).split('[')
            subtitle = [subtitle[9].split("latitude: ")[1][:~1], subtitle[10].split("longitude: ")[1][:~1]]

            # Untertitel hinzufügen
            text_position = (50, 100)
            cv2.putText(frame, f"Lat: {subtitle[0]}, Long: {subtitle[1]}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA) 

            if frame_counter >= switch_frame_number:
                cv2.putText(frame, f"GEAENDERT bei: Frame: {switch_frame_number}, Lat: {switch_subtitle[0]}, Long: {switch_subtitle[1]}", (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                x, y, w, h = bounding_boxes[frame_counter+1] 
                cv2.rectangle(frame, (x + w//4, y + h//4), (x + w//4 + w, y + h//4 + h), (0, 0, 255), 2)
            else:

                x, y, w, h = bounding_boxes[frame_counter+1] 
                cv2.rectangle(frame, (x + w//4, y + h//4), (x + w//4 + w, y + h//4 + h), (0, 255, 0), 2)
        

            # Frame speichern
            out.write(frame)
            frame_counter += 1
        except Exception as ex:
            pass
    # Alles freigeben
    cap.release()
    out.release()

# Beispielaufruf
generate_video_with_subtitles(video_path, output_video, switch_frame)


### Prüfung des Kurses

Zur Überprüfung des Kurses werden die Formeln zur Besteckrechnung nach Mittelbreite aus der Nautischen Formelsammlung Navigation[[6]](https://www.rolfdreyer.de/downloads/Formelsammlung.pdf) von Prof. Werner Huth verwendet. Der Kurs berechnet sich wie folgt: 

$$ tan(a) = \frac{\Delta\lambda \times \cos(\varphi_m)}{\Delta\varphi} $$

Wobei $ \Delta\lambda $ den Längenunterschied $ \lambda_B - \lambda_A $ und $ \Delta\varphi $ den Breitenunterschied $ \varphi_B - \varphi_A $ darstellen. Die mittlere Breite $ \varphi_m $ berechnet sich folgendermaßen:

$$ \varphi_m = \frac{1}{2} \times (\varphi_A + \varphi_B) $$

In den folgenden Code-Zellen werden diese implementiert. 

In [ ]:
def mittlere_breite(breite_a, breite_b):
    return 0.5 * (breite_a + breite_b)

def kurs_berechnen(laenge_a, laenge_b, breite_a, breite_b):
    breite_a = math.radians(float(breite_a))
    breite_b = math.radians(float(breite_b))
    laenge_a = math.radians(float(laenge_a))
    laenge_b = math.radians(float(laenge_b))
    
    laengen_unterschied = laenge_b - laenge_a
    breiten_unterschied = breite_b - breite_a
    
    kurs = math.atan2(
        laengen_unterschied * math.cos(mittlere_breite(breite_a, breite_b)),
        breiten_unterschied
    )

    kurs = math.degrees(kurs)
    
    if kurs < 0:
        kurs += 360
        
    return kurs  

Die für die Berechnung des Kurses benötigten Werte werden in der folgenden Zelle in ein Array von Dictionaries zusammengefasst. Ein Eintrag in diesem Array entspricht einen Videoausschnitt. Zu diesen Werten gehören der Name des Videos, der Untertitel in dem die Positionsdaten hinterlegt sind, die Frame-Indices der potenziellen Sektorengrenzen, sowie der Soll-Kurs der Sektorgrenze. Für eine weitere Evaluation liegen die Durchschnittswerte der Farbkanäle mit bei. 

In [ ]:
videoData = []

for i in range(1, 7):
    if i == 6:
        flug = 2
        part = 1
    else:
        flug = 1
        part = i
    
    kurs = 24    
    
    if i == 5:
        kurs = 320
    
    path = "data/Flug_" + str(flug) + "_part_" + str(part)

    bounding_boxes = get_Bounding_Boxes(path + ".MP4")
    frames_cropped, subtitles = get_all_frames_cropped(path + ".MP4", path + ".SRT", np.array(bounding_boxes), 400, 400, True)
    
    averages_hsv = np.zeros((frames_cropped.shape[0], frames_cropped.shape[3]), np.dtype('float32'))
    averages_rgb = averages_hsv.copy()
    
    for i in range(frames_cropped.shape[0]):
        averages_hsv[i] = get_avg_hsv(frames_cropped[i])
        averages_rgb[i] = get_avg_rgb(frames_cropped[i])

    indices_hsv = getIndices(averages_hsv)
    indices_rgb = getIndices(averages_rgb)
    
    videoData.append({
        "name": "Flug " + str(flug) + " Part " + str(part),
        "subtitles": subtitles,
        "averages_hsv": averages_hsv,
        "averages_rgb": averages_rgb,
        "indices_hsv": indices_hsv,
        "indices_rgb": indices_rgb,
        "kurs_soll": kurs
    })

Für die Berechnung des Kurses und der Entfernung zum Sektorenfeuer ist zunächst eine Umrechnung der vorliegenden Breiten- und Längengraden des Sektorenfeuers von ganzzahligen Grad, Minuten und Sekunden in Grad als Dezimalzahl erforderlich. Aus dem Untertitel des Frames, in dem die Sektorgrenze zu erkennen ist, werden die Breiten- und Längengrade der Drohne entnommen. Anhand der Position der Drohne und des Sektorenfeuers wird mittels der Funktion `kurs_berechnen` der Kurs ermittelt. Die Berechnung der Entfernung erfolgt mittels des Pakets `geopy`. 

In [ ]:
breitengrad_sektorenfeuer = 53 + 20/60 + 10/3600
laengengrad_sektorenfeuer = 7 + 6/60 + 30/3600

for video in videoData:
    frame_index_sektorgrenze = getIndices(video['averages_rgb'])[0]
    subtitles_frame = video['subtitles'][frame_index_sektorgrenze]

    breitengrad_drohne = float(subtitles_frame.split("latitude: ")[1][:~1].split("]")[0])
    laengengrad_drohne = float(subtitles_frame.split("longitude: ")[1][:~1].split("]")[0])

    kurs = kurs_berechnen(
        laengengrad_drohne, laengengrad_sektorenfeuer,
        breitengrad_drohne, breitengrad_sektorenfeuer,
    )
        
    entfernung = distance.distance(
        (breitengrad_drohne, laengengrad_drohne), 
        (breitengrad_sektorenfeuer, laengengrad_sektorenfeuer)
    ).nautical

    print(f'Video {video["name"]}')
    print(f'Position: N {np.round(breitengrad_drohne, 5)}° - E {np.round(laengengrad_drohne, 5)}°, Entfernung: {np.round(entfernung, 3)}sm')
    print(f'Der Kurs beträgt {np.round(kurs, 2)}° bei Frame {frame_index_sektorgrenze}, Abweichung von {np.round(kurs - video["kurs_soll"], 2)}°')
    print()

## Evaluation

| Video         | Erwarteter Kurs | Frame Sektorgrenze | Frame berechnet | Kurs berechnet | Differenz | Entfernung |
|---------------|-----------------|--------------------|-----------------|----------------|-----------|------------|
| Flug 1 Part 1 | 24°             | 84                 | 83              | 28.55°         | 4.55°     | 0.048sm    |
| Flug 1 Part 2 | 24°             | 113                | 111             | 28.77°         | 4.77°     | 0.048sm    |
| Flug 1 Part 3 | 24°             | 106                | 105             | 28.85°         | 4.85°     | 0.048sm    |
| Flug 1 Part 4 | 24°             | 64                 | 63              | 27.76°         | 3.76°     | 0.114sm    |
| Flug 1 Part 5 | 320°            | Nicht zu erkennen  | 86              | 323.05°        | 3.05°     | 0.110sm    |
| Flug 2 Part 1 | 24°             | 83                 | 82              | 27.16°         | 3.16°     | 0.213sm    |

In der oberen Tabelle sind der erwartete Kurs für die Sektorgrenze, sowie die manuell ermittelten Frames auf denen der Übergang zwischen zwei Sektoren zu erkennen ist. Im fünften Abflug ist kein eindeutiger Übergang zu erkennen, da zum Zeitpunkt des Übergangs die Kamera der Drohne ihre Richtung ändert. Der berechnete Frame 86 befindet sich jedoch in einem Bereich um diesen Zeitpunkt. 

<p align="center">
<img src="./abbildungen/wybelsum_kurs_flug_1_4.png" width="900">
<br>
Abbildung 3: Flug 1, Part 4. Eingezeichneter Kurs
</p>

Der berechnete Kurs weicht vom erwarteten Kurs in etwa um drei bis fünf Grad ab. Die Ursachen für diese Abweichung können in ungenauen Positionsdaten der Drohne liegen, oder auch in einer tatsächlichen Abweichung der Sektorgrenze. Zu erkennen ist, dass die Abweichung sich bei größeren Abständen verringert, was für eine Abweichung in den Positionsdaten der Drohne spricht. Die Abweichung ist in dem in Abbildung 3 eingezeichneten Kurs zu erkennen. Die Differenz zwischen den hier berechneten Kurs und den vom OpenSeaMap berechneten Kurs lässt sich durch Rundungsfehler erklären, da die Position in OpenSeaMap selbst mit ganzen Sekunden angegeben ist.  


In [ ]:
def plot_hsv_rgb(averages_hsv, indices_hsv, averages_rgb, indices_rgb, title):
    fig, axs = plt.subplots(3, 2, figsize=(15, 15))

    fig.suptitle(title, fontsize=16)

    axs[0][0].axvline(x = indices_hsv[0], color = 'b')
    axs[0][0].plot(averages_hsv[:, 0])
    axs[0][0].set_title("H Kanal")
    axs[0][0].set_xlabel("Frame")
    axs[1][0].axvline(x = indices_hsv[1], color = 'b')
    axs[1][0].plot(averages_hsv[:, 1])
    axs[1][0].set_title("S Kanal")
    axs[1][0].set_xlabel("Frame")
    axs[2][0].axvline(x = indices_hsv[2], color = 'b')
    axs[2][0].plot(averages_hsv[:, 2])
    axs[2][0].set_title("V Kanal")
    axs[2][0].set_xlabel("Frame")

    axs[0][1].axvline(x = indices_rgb[0], color = 'b')
    axs[0][1].plot(averages_rgb[:, 0])
    axs[0][1].set_title("Rot")
    axs[0][1].set_xlabel("Frame")
    axs[1][1].axvline(x = indices_rgb[1], color = 'b')
    axs[1][1].plot(averages_rgb[:, 1])
    axs[1][1].set_title("Grün")
    axs[1][1].set_xlabel("Frame")
    axs[2][1].axvline(x = indices_rgb[2], color = 'b')
    axs[2][1].plot(averages_rgb[:, 2])
    axs[2][1].set_title("Blau")
    axs[2][1].set_xlabel("Frame")
    plt.show()

for video in videoData:
    plot_hsv_rgb(video['averages_hsv'], video['indices_hsv'], video['averages_rgb'], video['indices_rgb'], video['name'])

Die oben gezeigten Diagramme zeigen den Verlauf der einzelnen Farbkanäle über die Zeit. Für eine Grenze zwischen roten und weißen Sektoren lassen sich mit den Farbkanälen H und S im HSV-Farbraum sowie Grün im RGB-Farbraum abweichende Indices für den Ziel-Frame ermitteln. Ebenso ist zu erkennen, dass im fünften Ausschnitt des ersten Fluges kein Erfassen des Leuchtfeuers erfolgt. Nach manueller Sichtung des Ausschnitts liegt der Grund hierfür, dass die Rücklichter eines parkenden Autos ebenso erfasst werden.  

## Fazit

Durch die erfolgte Bestimmung von Sektorgrenzen mittels Drohnenaufnahmen ist der Abschluss dieses Projekts als erfolgreich zu sehen. Das Projekt zeigt mögliche Implementierungen zur Bestimmung der Position des Sektorenfeuers in einem Video sowie den Zeitpunkt des Wechsels zwischen zwei Sektoren. Anhand der im Untertitel beiliegenden Metadaten lässt sich die Position der Drohne bestimmen, um somit den Kurs zu berechnen und diesen mit dem Erwartungswert gegenüberzustellen. Die Genauigkeit der Daten ist aus Sicht der Autoren den Umständen entsprechend zufriedenstellend. Möglichkeiten zur verbesserung der Messwerte liegen in der Verwendung eines genaueren GPS-Moduls für die Drohne, sowie eine Vergrößerung des Abstands zwischen der Drohne und des Sektorenfeuers. Für die Verwendung der in dieser Arbeit gezeigten Methoden bedarf es allerdings noch weitere Feinabstimmungen in der Erfassung des Leuchtfeuers. Neben der Verbesserung der Ergebnisse können zukünftige Projekte, welche auf diese Arbeit aufbauen, die Erfassung und Übergänge andersfarbiger Sektoren sowie Sektoren mit unterschiedlichen Kennungen, wie beispielsweise Blinkfeuer. 

## Literaturverzeichnis

- [1] Sportbootführerschein See kompakt https://doi.org/10.24053/9783739882161
- [2] Deutsche Leuchtfeuer: Leuchtturm Campen https://www.deutsche-leuchtfeuer.de/nordsee/campen.html
- [3] Deutsche Leuchtfeuer: Leuchtfeuer Wybelsum https://www.deutsche-leuchtfeuer.de/binnen/ems/wybelsum.html
- [4] OpenSeaMap https://map.openseamap.org/
- [5] Traffic light detection with color and edge information https://ieeexplore.ieee.org/document/5234518
- [6] Nautischen Formelsammlung Navigation https://www.rolfdreyer.de/downloads/Formelsammlung.pdf
- [7] Influence of normalization and color space to color texture classification https://www.sciencedirect.com/science/article/pii/S0031320316301510
